# Semesterlisten rekursiv ordnen

In dieser Übung wollen wir uns folgendes Problem ansehen: Gegeben ist eine Tabelle mit Semesterdaten. Attribute sind der Name des Semesters ("SS21"), die Datumsangaben des Beginns und des Endes des Semesters, respektive Prüfungszeiträume und ein Flag, das das aktuelle Semester anzeigt. Ein vergangenes Semester hat den Attributwert 0, das aktuelle Semester den Attributwert 1 und zukünftige Semester den Attributwert 2.

In [ ]:
# Hier ist nur Code zum Initialisieren der Umgebeung, bitte gehen Sie weiter, es gibt nichts zu sehen.

# Keine langen Fehlermeldungen
import sys
ipython = get_ipython()

def exception_handler(exception_type, exception, traceback):
    print("%s: %s" % (exception_type.__name__, exception), file=sys.stderr)

ipython._showtraceback = exception_handler

# Lade die Erweiterung, damit wir SQL Befehle nutzen können
%reload_ext sql
%sql sqlite://
%sql PRAGMA foreign_keys = ON

### Tabellen und Daten anlegen:

In [ ]:
%%sql

DROP TABLE IF EXISTS tblsemester;

CREATE TABLE tblsemester (
    pkSemesterID identity PRIMARY KEY,
    beginn DATE,
    beginn_pruefung DATE,
    ende DATE,
    ende_pruefung DATE,
    flagAktuell INTEGER not null,
    semesterbezeichnung TEXT,
    fkSemesterID INTEGER);

INSERT INTO `tblsemester` (pkSemesterID, fkSemesterID,semesterbezeichnung,beginn,ende,beginn_pruefung,ende_pruefung,flagAktuell)
VALUES 
 (1031, NULL, 'WS 19/20', '2019-10-01', '2020-01-22', '2020-01-26', '2020-02-15', 0),
 (1032, 1031, 'SS 2020', '2020-04-20', '2020-07-17', '2020-07-18', '2020-08-01', 0),
 (1033, 1032, 'WS 20/21', '2020-10-01', '2021-01-22', '2021-01-26', '2021-02-13', 0),
 (1034, 1033, 'SS 2021', '2021-03-15', '2021-07-09', '2021-07-12', '2021-07-31', 1),
 (1035, 1034, 'WS 21/22', '2021-10-01', '2022-01-21', '2022-01-22', '2022-02-15', 2);

### Daten Auslesen

Lassen wir uns alle Daten anzeigen, so finden wir folgendes:

In [ ]:
%sql SELECT * from tblsemester t;

# Aufgabe 1
Die einfachste Lösung ein Semester und alle seine Vorgänger zu finden, **unter der Annahme, dass die Ordnung durch den Primary Key korrekt ist**, ist einfach auf den Wert des Flags zu testen und zu sortieren:

# Rekursive SQL Anfragen

Eine Rekursion beginnt ist typischerweise so aufgebaut, dass eine Funktion erneut aufgerufen wird, falls eine Bedingung nicht zutrifft. Im Falls von SQL Rekursionen finden wir eine Tabelle vor, in der ein Foreign Key auf den Primary Key der eigenen Tabelle verweist. Diese Tabelle muss dann mit sich selbst ge-`JOIN`-t werden. Es beginnt also wieder mit einem Datensatz der Tabelle und einer (im voraus unbekannten Anzahl) von `JOIN`s mit sich selbst. Das Abbruchkriterium ist wenn keine weiteren Ergebnisse gefunden werden. Das Startkriterium ist eine Zeile der Tabelle aus der die Rekursion startet. Die Datensätze müssen miteinander verbunden werden, das ist die aufgabe einens `UNION` Befehls.

### UNION
Union verbindet die Ergebnisse mehrerer SELECT Befehle. Dazu müssen die SELET befehle die selbe Anzahl an Spalten und Datentypen zurückgeben und diese müssen auch gleich angeordnet sein. `UNION ALL` gibt alle Ergebnisse zurück, ohne das `ALL` werden Duplikate entfernt.

### Vorgesetze als Beispiel:
Ein typisches Beispiel für rekursive Abfragen sind Abteilungshierarchien. Bauen wir uns für diese Übung eine "Chain of Command" auf:

In [ ]:
%%sql 
DROP TABLE IF EXISTS Personen;
CREATE TABLE Personen(
    pkPerson INTEGER PRIMARY_KEY UNIQUE,
    nachname TEXT,
    vorname TEXT, 
    vorgesetzter INTEGER FOREIGN_KEY REFERENCES Personen(pkPerson)
);
INSERT INTO PERSONEN VALUES (1, 'Strauß', 'Peter', NULL), (2, 'Huber', 'Andreas', 1), (3, 'Moser', 'Udo', 1), (4, 'Elser', 'Benedikt', 3), (5, 'Jobs', 'Jürgen', 1), (6, 'Gates', 'Christian', 5);

Wer ist nun der Vorgesetzte von Herrn Elser?

In [ ]:
%%sql
SELECT
  p1.nachname, p1.vorname, '<' as sticht, p2.nachname, p2.vorname
FROM
  Personen p1
LEFT JOIN Personen p2 ON(p2.pkPerson = p1.vorgesetzter)
WHERE p1.Nachname = 'Elser';

Aber wie kann man diese "Chain of Command" nach ganz oben auflösen? Man müsste ja aus einer Tabelle auswählen, die automatisch den `JOIN` auf sich selbst macht. 

Dazu verwendet man eine *rekursive Abfrage*. Diese temporäre Tabelle (CTE) bekommt als erstes einen Namen: `WITH RECURSIVE <tabellenname>`. Die Definition dieser Tabelle besteht aus einer initialen Abfrage (in dieser bestimmen wir den Datensatz von Herrn Elser) und einem rekursiven Anteil in dem wir auf die gerade definierte Tabelle (`<tabellenname>`) zugreifen. Dort hangeln wir uns vom initialen Datensatz aus weiter.

Konkret bauen unsere Abfrage wie folgt auf:

In [ ]:
%%sql
WITH RECURSIVE untergeben(nachname, vorname, pkPerson, vorgesetzter) AS (
   --- Initiale Abfrage
   SELECT
     p.nachname, p.vorname, p.pkPerson, p.vorgesetzter
   FROM
     Personen p
     WHERE nachname = "Elser"
   UNION
   --- Rekursive Abfrage
   SELECT
     p1.nachname, p1.vorname, p1.pkPerson, p1.vorgesetzter
   FROM
     Personen p1, untergeben v
   WHERE
     p1.pkPerson = v.vorgesetzter
) SELECT * from untergeben;

Wir haben also eine Tabelle namens `untergeben` in einer Common table expression definiert. Innerhalb der Definition (`AS ( ... )` beginnen wir mit einer initialen Abfrage in der wir uns **nicht** auf die CTE `untergeben` beziehen. Wir suchen vorerst nur den Datensatz von dem aus die Rekursion startet aus der Tabelle `Personen`. Im zweiten Teil, der rekursiven Abfrage machen wir einen impliziten `JOIN` zwischen der Tabelle `Personen` und der CTE `untergeben`. Diese Rekursion endet wenn keine weiteren Ergebnisse vorhanden sind, also wenn der Vorgesetzte von Herrn Strauß `NULL` ist. 

Rekursiv kann ich aber auch die andere Richtung beantworten: Wem ist ein Nutzer vorgesetzt? Auch hier wieder ein initialer Datensatz, dann geht es rekursiv weiter.

In [ ]:
%%sql
WITH RECURSIVE vorgesetzt(nachname, vorname, pkPerson, vorgesetzter) AS (
   --- Initiale Abfrage
   SELECT
     p.nachname, p.vorname, p.pkPerson, p.vorgesetzter
   FROM
     Personen p
     WHERE nachname = "Strauß"
   UNION
   --- Rekursive Abfrage
   SELECT
     p1.nachname, p1.vorname, p1.pkPerson, p1.vorgesetzter
   FROM
     Personen p1, vorgesetzt v
   WHERE
     p1.vorgesetzter = v.pkPerson
) SELECT * from vorgesetzt;

Grafisch ist das wie folgt [in diesem Medium Artiel](https://medium.com/swlh/recursion-in-sql-explained-graphically-679f6a0f143b) aufbereitet:

![](https://miro.medium.com/max/2400/1*Jn69R88TYbuJEP-qrF91Dg.png)

# Aufgabe 2

Wollen wir sichergehen, dass die rekursive Reihe eingehalten wird nutzen wir eine rekursive Abfrage. 

- Starten Sie in der initialen Abfrage, indem Sie den Beginn der Kette suchen. *Hinweis: Vorne ist wo auf kein früheres Semester verwiesen wird*.
- Im Rekursiven Teil müssen Sie wie in Aufgabe 1 bremsen, wenn das flagAktuell zu groß wird. 
- In der eigentlichen Abfrage müssen Sie lediglich abfragen, oder Sie filtern hier auf flagAktuell

# Ein echtes Flag im Datenmodell
Sind nun die Daten aber anders aufgebaut, so dass es nur noch ein Flag aktuell Ja/Nein gibt wird es schwieriger:

In [ ]:
%%sql

DROP TABLE tblsemester;

CREATE TABLE tblsemester (
    pkSemesterID identity PRIMARY KEY,
    beginn DATE,
    beginn_pruefung DATE,
    ende DATE,
    ende_pruefung DATE,
    flagAktuell INTEGER not null,
    semesterbezeichnung TEXT,
    fkSemesterID INTEGER);

INSERT INTO `tblsemester` (pkSemesterID, fkSemesterID,semesterbezeichnung,beginn,ende,beginn_pruefung,ende_pruefung,flagAktuell)
VALUES 
 (1031, NULL, 'WS 19/20', '2019-10-01', '2020-01-22', '2020-01-26', '2020-02-15', 0),
 (1032, 1031, 'SS 2020', '2020-04-20', '2020-07-17', '2020-07-18', '2020-08-01', 0),
 (1033, 1032, 'WS 20/21', '2020-10-01', '2021-01-22', '2021-01-26', '2021-02-13', 0),
 (1034, 1033, 'SS 2021', '2021-03-15', '2021-07-09', '2021-07-12', '2021-07-31', 1),
 (1035, 1034, 'WS 21/22', '2021-10-01', '2022-01-21', '2022-01-22', '2022-02-15', 0);

Der Naive Ansatz funktioniert nun nicht mehr:

In [ ]:
%%sql
SELECT pkSemesterID, semesterbezeichnung, fkSemesterID, flagAktuell
FROM tblsemester
WHERE flagAktuell <= 1
ORDER BY pkSemesterID;

# Aufgabe 3

Prüfen Sie, ob Ihre Lösung noch funktioniert und formulieren Sie Ihren Ansatz ggf. um, falls es zu Problemen kommt.

# Aufgabe 4

Selbstverständlich sind die pkSmesterIDs hier perfekt in Reihenfolge. Ändern Sie mit einem Update Befehl den pk (und die fk(s)!) und prüfen Sie, ob Ihre Abfrage auch wirklich noch korrekt läuft.